In [2]:
import sys
!{sys.executable} -m pip install qdrant-client ollama sqlalchemy


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv(os.path.join(os.getcwd(),'input/20251115-mfi-products.csv'), delimiter=';')
# Strip whitespace from string columns
df = df.map(lambda x: x.strip() if isinstance(x, str) else x).fillna('null')
df.head()

,product_name_alias,product_description,shelflife,CN_code,country_of_origin,product_group,product_segment_text,dairy,bakery_sweets,nutrition,...,rapeseed_fat,hydrogenated_fat,non_hydrogenated_fat,fat_content_below_40,fat_content_between_40_and_70,fat_content_between_70_and_80,protein_content_below_20,protein_content_between_20_and_50,protein_content_between_50_and_70,protein_content_between_70_and_95
0,Product 101,Casein,36 months,3501 1090,DE,Casein,"dairy,bakery & sweets,nutrition,savoury,single...",True,True,True,...,False,False,False,False,False,False,False,False,False,True
1,Product 102,Casein,36 months,3501 1090,DE,Casein,"dairy,bakery & sweets,nutrition,savoury,single...",True,True,True,...,False,False,False,False,False,False,False,False,False,True
2,Product 103,Casein,36 months,3501 1090,DE,Casein,"dairy,bakery & sweets,nutrition,savoury,single...",True,True,True,...,False,False,False,False,False,False,False,False,False,True
3,Product 104,Casein,36 months,3501 1090,DE,Casein,"dairy,bakery & sweets,nutrition,savoury,single...",True,True,True,...,False,False,False,False,False,False,False,False,False,True
4,Product 105,"Hydrogenated dairy oil 40%, includes SWP and L...",18 months,1901 9099,DE,Fat Powder,bakery & sweets,False,True,False,...,False,True,False,False,True,False,False,False,False,False


In [4]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

res = df.to_sql("products", con=engine, if_exists="replace", index=False)
print(res)

2025-11-15 17:10:24,640 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-15 17:10:24,642 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("products")
2025-11-15 17:10:24,642 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-15 17:10:24,643 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("products")
2025-11-15 17:10:24,643 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-15 17:10:24,644 INFO sqlalchemy.engine.Engine 
CREATE TABLE products (
	product_name_alias TEXT, 
	product_description TEXT, 
	shelflife TEXT, 
	"CN_code" TEXT, 
	country_of_origin TEXT, 
	product_group TEXT, 
	product_segment_text TEXT, 
	dairy BOOLEAN, 
	bakery_sweets BOOLEAN, 
	nutrition BOOLEAN, 
	savoury BOOLEAN, 
	single_ingredients BOOLEAN, 
	kosher BOOLEAN, 
	super_kosher BOOLEAN, 
	halal BOOLEAN, 
	vlog BOOLEAN, 
	bio BOOLEAN, 
	rspo_mb BOOLEAN, 
	rspo_sg BOOLEAN, 
	vegetarian BOOLEAN, 
	vegan BOOLEAN, 
	palmfree BOOLEAN, 
	clean BOOLEAN, 
	acid_stable BOOLEAN, 
	palm_fat BOOLEAN, 
	dairy_fat_

In [5]:
import json

# Create knowledge dictionary with table schema information
knowledge = {}
knowledge["object_type"] = "table"
knowledge["table_name"] = "products"


def f(v):
    if isinstance(v, tuple):
        return str(v[0])
    return v


def get_column_info(df, column):
    column_info = {}
    column_info["dtype"] = df[column].dtype,
    column_info["non_null_count"] = df[column].count(),
    column_info["null_count"] = df[column].isnull().sum(),
    column_info["unique_values"] = df[column].nunique(),
    column_info["sample_values"] = df[column].dropna().unique()[:5].tolist(),
    column_info["statistics"] = df[column].describe().to_dict() if pd.api.types.is_numeric_dtype(df[column]) else None
    return {k: f(v) for k, v in column_info.items()}


# Generate schema information in JSON format and add to knowledge
schema_info = {}

for column in df.columns:
    schema_info[column] = get_column_info(df, column)

knowledge["schema"] = schema_info

# Get sample rows from the DataFrame and add to knowledge
sample_rows = df.sample(2, random_state=1)

knowledge["sample_rows"] = sample_rows.to_dict(orient="records")

with open("knowledge.json", "w") as f:
    json.dump(knowledge, f)
    f.close()


In [6]:
# Read and pretty-print the knowledge JSON file
with open("knowledge.json", "r") as f:
    parsed = json.load(f)
    print(json.dumps(parsed, indent=4))
    f.close()

{
    "object_type": "table",
    "table_name": "products",
    "schema": {
        "product_name_alias": {
            "dtype": "object",
            "non_null_count": "137",
            "null_count": "0",
            "unique_values": "137",
            "sample_values": "['Product 101', 'Product 102', 'Product 103', 'Product 104', 'Product 105']",
            "statistics": null
        },
        "product_description": {
            "dtype": "object",
            "non_null_count": "137",
            "null_count": "0",
            "unique_values": "102",
            "sample_values": "['Casein', 'Hydrogenated dairy oil 40%, includes SWP and Lactose, milky taste, freeze and thaw stable', 'Hydrogenated dairy oil 60%, includes SWP and Lactose, milky taste, freeze and thaw stable, high whitening power', 'Milk Protein Compound', 'High protein ~74% + lactose, egg replacer in bakery applications']",
            "statistics": null
        },
        "shelflife": {
            "dtype": "object",

In [7]:
import json

# Select the relevant columns for the query
# selected_columns = ['product_name', 'product_description', 'product_group', 'product_segment_text', 'palmfree', 'vegan', 'vegetarian']

df_copy = df.copy()

# Clean up the DataFrame by dropping rows with NaN values
df_copy.dropna(inplace=True)

# Strip whitespace from string columns
df_copy = df_copy.map(lambda x: x.strip() if isinstance(x, str) else x)

# Convert the DataFrame to a JSON string for each row
df_copy['json_data'] = df_copy.apply(lambda row: json.dumps(row.to_dict(), ensure_ascii=False), axis=1)

# print the first few rows of the cleaned DataFrame
df_copy

,product_name_alias,product_description,shelflife,CN_code,country_of_origin,product_group,product_segment_text,dairy,bakery_sweets,nutrition,...,hydrogenated_fat,non_hydrogenated_fat,fat_content_below_40,fat_content_between_40_and_70,fat_content_between_70_and_80,protein_content_below_20,protein_content_between_20_and_50,protein_content_between_50_and_70,protein_content_between_70_and_95,json_data
0,Product 101,Casein,36 months,3501 1090,DE,Casein,"dairy,bakery & sweets,nutrition,savoury,single...",True,True,True,...,False,False,False,False,False,False,False,False,True,"{""product_name_alias"": ""Product 101"", ""product..."
1,Product 102,Casein,36 months,3501 1090,DE,Casein,"dairy,bakery & sweets,nutrition,savoury,single...",True,True,True,...,False,False,False,False,False,False,False,False,True,"{""product_name_alias"": ""Product 102"", ""product..."
2,Product 103,Casein,36 months,3501 1090,DE,Casein,"dairy,bakery & sweets,nutrition,savoury,single...",True,True,True,...,False,False,False,False,False,False,False,False,True,"{""product_name_alias"": ""Product 103"", ""product..."
3,Product 104,Casein,36 months,3501 1090,DE,Casein,"dairy,bakery & sweets,nutrition,savoury,single...",True,True,True,...,False,False,False,False,False,False,False,False,True,"{""product_name_alias"": ""Product 104"", ""product..."
4,Product 105,"Hydrogenated dairy oil 40%, includes SWP and L...",18 months,1901 9099,DE,Fat Powder,bakery & sweets,False,True,False,...,True,False,False,True,False,False,False,False,False,"{""product_name_alias"": ""Product 105"", ""product..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,Product 233,Milk Protein Compound,18 months,0404 1014,DE,Milk Protein Compound,dairy,True,False,False,...,False,False,False,False,False,False,False,False,False,"{""product_name_alias"": ""Product 233"", ""product..."
133,Product 234,"Protein ~70%, stirred and set yoghurt, sour cream",18 months,2106 9098,DE,Milk Protein Compound,dairy,True,False,False,...,False,False,False,False,False,False,False,False,False,"{""product_name_alias"": ""Product 234"", ""product..."
134,Product 235,"Protein ~85%, stirred yoghurt",18 months,3504 0010,DE,Milk Protein Compound,dairy,True,False,False,...,False,False,False,False,False,False,False,False,False,"{""product_name_alias"": ""Product 235"", ""product..."
135,Product 236,"Milk protein 86%, clarification of liquides, f...",36 months,3501 9090,DE,Milk Protein Compound,"dairy,savoury,single ingredients",True,False,False,...,False,False,False,False,False,False,False,False,False,"{""product_name_alias"": ""Product 236"", ""product..."


In [8]:
from modules import vectorizer

collection_name = "test_collection"
# vectorizer.something([df_copy, df_knowledge], collection_name=collection_name)

# dict_knowledge_vec = vectorizer.generate_embeddings(df_knowledge['json'].iloc[0])

df_copy_short = df_copy[['product_name_alias', 'product_description', 'json_data']].copy()
df_copy_short['embedding'] = df_copy_short['json_data'].apply(vectorizer.generate_embeddings)
df_copy_short

,product_name_alias,product_description,json_data,embedding
0,Product 101,Casein,"{""product_name_alias"": ""Product 101"", ""product...","[-0.27703148126602173, -0.2753241956233978, 0...."
1,Product 102,Casein,"{""product_name_alias"": ""Product 102"", ""product...","[-0.2922098636627197, -0.272165447473526, 0.53..."
2,Product 103,Casein,"{""product_name_alias"": ""Product 103"", ""product...","[-0.2887406647205353, -0.31097880005836487, 0...."
3,Product 104,Casein,"{""product_name_alias"": ""Product 104"", ""product...","[-0.2671581506729126, -0.2773367762565613, 0.5..."
4,Product 105,"Hydrogenated dairy oil 40%, includes SWP and L...","{""product_name_alias"": ""Product 105"", ""product...","[-0.4954999089241028, -0.49323081970214844, 0...."
...,...,...,...,...
132,Product 233,Milk Protein Compound,"{""product_name_alias"": ""Product 233"", ""product...","[-0.23961184918880463, -0.3557477593421936, 0...."
133,Product 234,"Protein ~70%, stirred and set yoghurt, sour cream","{""product_name_alias"": ""Product 234"", ""product...","[-0.3258049190044403, -0.4063526690006256, 0.4..."
134,Product 235,"Protein ~85%, stirred yoghurt","{""product_name_alias"": ""Product 235"", ""product...","[-0.2478567212820053, -0.39719828963279724, 0...."
135,Product 236,"Milk protein 86%, clarification of liquides, f...","{""product_name_alias"": ""Product 236"", ""product...","[-0.38761067390441895, -0.08961507678031921, 0..."


In [9]:
from modules import qdrant

res = qdrant.store_vectors_in_qdrant(df_copy_short, ["product_name_alias", "product_description"] , collection_name)
print(res)

Stored 137 vectors in Qdrant collection 'test_collection'. Operation info: operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [10]:
from modules import generator, qdrant, vectorizer

# Not all queries work as expected!

# search_term = "which product is a whipping agent?"
# search_term = "which product is only good for bakery & sweets?"
# search_term = "which product is good for dairy products?"
# search_term = "70% fat" # failed
# search_term = "which product contains less than 71% fat?" # failed
# search_term = "which product is a whey powder?"
# search_term = "which product is a fatpowder?" # failed
# search_term = "which product is good for decoration?"
# search_term = "which product is good for decoration and beverages?" #failed
# search_term = "which product is vegan and for savoury use?" #failed

from modules.input import get_user_input, validate_search_term

# # Get user input
# search_term = get_user_input("Enter search term")
# if validate_search_term(search_term):
#     # Proceed with search
#     pass

# search_result = qdrant.perform_search_in_qdrant(
#     query = vectorizer.generate_embeddings(search_term),
#     collection_name = collection_name,
#     limit = 1
# )


# for i in search_result:
#     print(f'{i.score}: {i.payload["product_name"]}, {i.payload["product_description"]}')


In [11]:
import ollama

collection_name = "test_collection"

def retrieve(query, collection_name2):
    result = qdrant.perform_search_in_qdrant(query, collection_name=collection_name2, limit=1)
    return f"{result[0].payload['product_name']}: {result[0].payload['product_description']}"


# template = "Answer the following question:"
question = "vegan and for nutrition use?"
prompt= f"You are an AI language model assistant. Your task is to find matching products to the {question} using only results from a product database with this schema: {knowledge}. Create an SQL query that returns the product name aliases and descriptions of the matching products."

models = ['mistral','stablelm2','avr/sfr-embedding-mistral']

answer = ollama.generate(
    model=models[0],
    prompt=prompt
)

print(answer['response'])

```sql
   SELECT product_name_alias, product_description
   FROM your_table_name
   WHERE product_group IN ('Fat Powder', 'Lactose')  -- Adjust this part based on the desired product group(s)
   ORDER BY product_name_alias;
   ```

This SQL query will return the product name aliases and descriptions for products that belong to the specified groups (in this case, Fat Powder and Lactose). You can adjust the `product_group` part of the WHERE clause to include other groups as needed.


In [12]:
answer = generator.generate(search_result,search_term,0,0)
print(answer['response'])

NameError: name 'search_result' is not defined